<h2> 0. 데이터 로드 

In [28]:
!unzip /Users/sungheui/코딩 테스트/ArtClassify/train.zip -d ./

unzip:  cannot find or open /Users/sungheui/코딩, /Users/sungheui/코딩.zip or /Users/sungheui/코딩.ZIP.


In [31]:
pwd

'/Users/sungheui/sh/lotteAI'

In [52]:
#환경 설정 확인하기
import torch
print(f"mps 사용 가능 여부: {torch.backends.mps.is_available()}")
print(f"mps 지원 환경 여부: {torch.backends.mps.is_built()}")

#GPU check DEVICE
# print(f'PyTorch Version : [{torch.__version__}]')
# device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(f'Device : [{device}]')

mps 사용 가능 여부: True
mps 지원 환경 여부: True


<h3> 2. 라이브러리 import 

In [53]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from glob import glob
import cv2
from PIL import Image
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
from torch.optim.lr_scheduler import ReduceLROnPlateau 

<h3> 3. Config setting

In [54]:
#CONFIG
torch.manual_seed(128)
BATCH_SIZE= 2
EPOCHS=40
LEARNING_RATE=1e-3
print(f'PyTorch Version : [{torch.__version__}]')
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"Device: {device}")

PyTorch Version : [2.1.2]
Device: mps


<h3> 4. Custom Datasets

In [68]:
# import pdb;pdb.set_trace() 
class LotteDataset(Dataset):
  def __init__(self, data_root, train_mode, transform=None):
    super(LotteDataset, self).__init__()
    self.train_mode=train_mode
    self.transform = transform
    self.label2idx = {'dog': 0,
             'elephant': 1,
             'giraffe': 2,
             'guitar': 3,
             'horse': 4,
             'house': 5,
             'person': 6}
   
    if self.train_mode==False:
        
        self.img_list= []
        img_list = []
        for file in os.listdir(data_root):
            file_root = os.path.join(data_root, file)
            for data in os.listdir(file_root):
                data_path = os.path.join(file_root, data)
                img_list.append(data_path)
        self.img_list = sorted(img_list)
        
    else: #학습할 때 
        self.img_list = []
        self.label_list=[]
        img_list = []
        for file in os.listdir(data_root):
            file_root = os.path.join(data_root, file)
            for data in os.listdir(file_root):
                data_path = os.path.join(file_root, data)
                img_list.append(data_path)
        self.img_list = sorted(img_list)

        for label in img_list:
            label = label.split('/')[3]
            self.label_list.append(self.label2idx[label]) 
            

  def __len__(self):
      return len(self.img_list)

  def __getitem__(self, index):
    img_path = self.img_list[index]
    
    if self.train_mode:
        label = self.label_list[index]
     

    # Image Loading
    img = Image.open(img_path)

    if self.transform:
      img = self.transform(img)

    if self.train_mode:
      return img,label
    else:
      return img

# if __name__=='__main__':
#     data = LotteDataset('./dataset/train',train_mode=True,transform=train_transforms)
#     q= data.__getitem__(1)
#     print(label)


In [69]:
class MapTransform(Dataset):
    def __init__(self, dataset, transform, train_mode):
        self.dataset = dataset
        self.transform=transform
        self.train_mode=train_mode

    def __getitem__(self, index):
        if self.train_mode:
          return self.transform(self.dataset[index][0]), self.dataset[index][1]
        else:
          return self.transform(self.dataset[index])

    def __len__(self):
        return len(self.dataset)

In [70]:
#Trian Augmentation 
train_transforms=transforms.Compose([
    transforms.RandomChoice([
        transforms.ColorJitter(brightness=(1,1.1)),
        transforms.ColorJitter(contrast=0.1), 
        transforms.ColorJitter(saturation=0.1),
    ]),
    transforms.RandomChoice([
        transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=10,fill=255),
        transforms.RandomCrop((224,224)),
    ]),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
#Test Augmentation 
test_transforms=transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224,224)),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

In [71]:
#그냥 train-test -> validation x
train_data=LotteDataset('./dataset/train',train_mode=True)
test_data=LotteDataset('./dataset/test',train_mode=False)

trans_train_data=MapTransform(train_data,train_transforms,train_mode=True)
trans_test_data=MapTransform(test_data,test_transforms,train_mode=False)

train_iter=DataLoader(trans_train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
test_iter=DataLoader(trans_test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)

In [72]:
##cross-validation 할때 이걸로 
# all_data=LotteDataset('./dataset/train',train_mode=True,transform=train_transforms)
# test_data=LotteDataset('./dataset/test',train_mode=False,transform=test_transforms)
# test_iter=DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)

In [73]:
print(len(trans_train_data))
print(len(test_data))

1699
351


<h3> 5. Model Load 

In [74]:
#현재 디렉토리 찾기 
# os.getcwd()

%reload_ext autoreload
%autoreload 2

from train.tt import VIT2

model = VIT2().to(device)

In [75]:
for name, param in model.named_parameters():
    if param.requires_grad==True:
        print(name)

prompt_embeddings
model.head.weight
model.head.bias


In [76]:
#loss label smooth
def loss_fn(outputs, targets):
    if len(targets.shape) == 1:
        return F.cross_entropy(outputs, targets)
    else:
        return torch.mean(torch.sum(-targets * F.log_softmax(outputs, dim=1), dim=1))

def label_smooth_loss_fn(outputs, targets, epsilon=0.1):
    onehot = F.one_hot(targets, 1000).float().to(device)
    targets = (1 - epsilon) * onehot + torch.ones(onehot.shape).to(device) * epsilon / 1000
    return loss_fn(outputs, targets)

In [ ]:
def func_eval(Model,data_iter,loss):
    with torch.no_grad():
      Model.eval()
      n_total, n_correct = 0,0
      loss_val_sum=0
      print("(Train or Validation) Data Testing....\n")
      for imgs, labels in iter(data_iter):
        imgs, labels = imgs.to(device), labels.to(device)
        
        model_pred=Model(imgs)

        loss_out=loss(model_pred,labels)
        loss_val_sum+=loss_out

        _, y_pred=torch.max(model_pred.data,1)
        n_correct+=(y_pred==labels).sum().item()
        n_total+=imgs.size(0)
      val_accr=(n_correct/n_total)
      Model.train()
      loss_val_avg=loss_val_sum/len(data_iter)
    print("Testing Done.\n")
    return val_accr,loss_val_avg
    
def get_submission(Model,data_iter,epoch,fold):
  with torch.no_grad():
    Model.eval()
    pred_label=[]
    print("Final Testing....\n")
    for imgs in iter(test_iter):
      model_pred=Model(imgs.to(device))

      _, y_pred=torch.max(model_pred.data,1)
      pred_label.extend(y_pred.tolist())

  Model.train()

  submission = pd.read_csv('./dataset/test_answer_sample_.csv', encoding = 'utf-8')
  submission['prediction'] = pred_label
  submission.to_csv('./submission_50Resnext'+str(fold)+'_'+str(epoch)+'.csv', index = False)

<h3> Train

<h2> cross validation 

In [80]:
folds=StratifiedKFold(n_splits=4,shuffle=True)
scaler = torch.cuda.amp.GradScaler()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
##Loss Function 택1
# loss=label_smooth_loss_fn
loss = loss_fn 
f = open("./vitPrompt_trainlog2.txt", 'w')



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [81]:
print(BATCH_SIZE)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(all_data,all_data.label_list)):
    
  train_data=torch.utils.data.Subset(all_data,train_idx)
  vali_data=torch.utils.data.Subset(all_data,vali_idx)
    
  train_iter=DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
  vali_iter=DataLoader(vali_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)

  print(f'::::::::: Fold : {current_fold} :::::::::::\n')
  f.write(f'::::::::: Fold : {current_fold} :::::::::::\n')
  # model.train()
  prev_vali_loss=100
  best_vali_loss=100
  flag=0
  over_check=0
  for epoch in range(EPOCHS) :
    # if epoch==3:
    #   freeze(Model,-3)
    loss_val_sum=0
    for imgs, labels in iter(train_iter):
        imgs, labels = imgs.to(device), labels.to(device)
        model_pred=model(imgs)
        loss_out = loss(model_pred, labels)
        scaler.scale(loss_out).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_val_sum+=loss_out
    loss_val_avg=loss_val_sum/len(train_iter)
    vali_accr,vali_loss=func_eval(model,vali_iter,loss_fn)
    if epoch>=0:
      get_submission(model,test_iter,epoch,current_fold)
      print("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      f.write("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      print("Model Save....\n")
      torch.save({'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()}, './checkpoint/'+str(current_fold)+'_VPT_epoch_'+str(epoch)+'.tar')
    else:
      print("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      f.write("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
    scheduler.step(vali_loss) # LR Scheduler

    if prev_vali_loss<vali_loss: #Stop Train
      flag+=1
      if flag==10 : 
        print("Stop Training...\n")
        break
    if best_vali_loss>vali_loss:
      flag=0
      best_vali_loss=vali_loss
    prev_vali_loss=vali_loss
f.close()

2
::::::::: Fold : 0 :::::::::::



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: 

In [119]:
dataloader_iter = iter(train_iter)
batch = next(dataloader_iter)
print(batch)

[tensor([[[[ 2.2489,  2.2489,  2.2489,  ..., -0.4910, -0.0598,  0.2982],
          [ 2.2489,  2.2489,  2.2489,  ..., -0.5508, -0.0687, -0.1299],
          [ 2.2489,  2.2489,  2.2489,  ..., -0.2422, -0.2603, -0.1485],
          ...,
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489]],

         [[ 2.4286,  2.4286,  2.4286,  ..., -0.4640,  0.0993,  0.5209],
          [ 2.4286,  2.4286,  2.4286,  ..., -0.5219,  0.0903,  0.0832],
          [ 2.4286,  2.4286,  2.4286,  ..., -0.1579, -0.1039,  0.0618],
          ...,
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286]],

         [[ 2.6400,  2.6400,  2.6400,  ..., -0.6255, -0.0623,  0.4076],
          [ 2.6400,  2.6400, 

mps
